In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import matplotlib.pyplot as plt
import pdb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
import itertools
import pdb
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
%matplotlib inline

In [36]:
colName = ['lettr','x-box','ybox','width','high','onpix','x-bar','y-bar','x2bar','y2bar','xybar','x2ybr','xy2br','x-ege','xegvy','y-ege','yegvx'];
df = pd.read_csv('/kaggle/input/letters-ex/letters_CG.txt',header = 0,names = colName,sep=' ');
df.head()

In [32]:
df.shape

In [33]:
df.info()

In [34]:
print(df.columns)

In [35]:
sequence = list(np.sort(df['lettr'].unique()))
print(sequence)

In [38]:
data_mean = df.groupby('lettr').mean()
data_mean.head()

In [40]:
X = df.drop(['lettr'],axis = 1)
y = df['lettr']

In [42]:
from sklearn.preprocessing import scale
X = scale(X)

In [44]:
from sklearn.model_selection import train_test_split

# Spliting the dataset into train-test
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.7,test_size=0.3,random_state=101)

In [45]:
from sklearn.svm import SVC

# Building a linear SVM model
linear_model = SVC(kernel='linear')
linear_model.fit(X_train,y_train)

y_pred = linear_model.predict(X_test)

In [46]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix

# accuracy and confusion matrix
print("accuracy :", metrics.accuracy_score(y_true = y_test,y_pred = y_pred), "\n")
print("confusion_matrix :", metrics.confusion_matrix(y_true = y_test,y_pred = y_pred))

In [50]:
#Building a non-linear SVM model
non_linear_model = SVC(kernel = 'rbf')
non_linear_model.fit(X_train,y_train)

# Predict
y_pred = non_linear_model.predict(X_test)
print("accuracy :", metrics.accuracy_score(y_true = y_test,y_pred = y_pred), "\n")
print("confusion_matrix :", metrics.confusion_matrix(y_true = y_test,y_pred = y_pred))

In [53]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
folds = KFold(n_splits = 5, shuffle = True, random_state = 101)
hyper_params = [ {'gamma': [1e-2, 1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]}]
model = SVC(kernel="rbf")
model_cv = GridSearchCV(estimator = model, 
                        param_grid = hyper_params, 
                        scoring= 'accuracy', 
                        cv = folds, 
                        verbose = 1,
                        return_train_score=True)   
model_cv.fit(X_train, y_train)   

In [54]:
GridSearchCV(cv=KFold(n_splits=5, random_state=101, shuffle=True),
             estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000],
                          'gamma': [0.01, 0.001, 0.0001]}],
             return_train_score=True, scoring='accuracy', verbose=1)

In [55]:
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results

In [57]:
cv_results['param_C'] = cv_results['param_C'].astype('int')

# # plotting
plt.figure(figsize=(16,6))

#  1/3
plt.subplot(131)
gamma_01 = cv_results[cv_results['param_gamma']==0.01]

plt.plot(gamma_01["param_C"], gamma_01["mean_test_score"])
plt.plot(gamma_01["param_C"], gamma_01["mean_train_score"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Gamma=0.01")
plt.ylim([0.60, 1])
plt.legend(['test accuracy', 'train accuracy'], loc='upper left')
plt.xscale('log')

#  2/3
plt.subplot(132)
gamma_001 = cv_results[cv_results['param_gamma']==0.001]

plt.plot(gamma_001["param_C"], gamma_001["mean_test_score"])
plt.plot(gamma_001["param_C"], gamma_001["mean_train_score"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Gamma=0.001")
plt.ylim([0.60, 1])
plt.legend(['test accuracy', 'train accuracy'], loc='upper left')
plt.xscale('log')
plt.subplot(133)
gamma_0001 = cv_results[cv_results['param_gamma']==0.0001]
#3/3
plt.plot(gamma_0001["param_C"], gamma_0001["mean_test_score"])
plt.plot(gamma_0001["param_C"], gamma_0001["mean_train_score"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Gamma=0.0001")
plt.ylim([0.60, 1])
plt.legend(['test accuracy', 'train accuracy'], loc='upper left')
plt.xscale('log')

In [5]:
pd.isnull(df).sum()

In [6]:
plt.subplots(figsize=(8,4))
sns.barplot(x = df['lettr'].value_counts().index, y = df['lettr'].value_counts().values);
plt.xlabel('Classes');
plt.ylabel('Frequency');